<a href="https://colab.research.google.com/github/Steven032/ECE1512-2023F-ProjectRepo-Xiaohu.Yang-Yixin.Feng/blob/main/Task2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project A: Task 2

reference

https://keras.io/guides/transfer_learning/\

https://keras.io/examples/vision/knowledge_distillation/

https://keras.io/guides/training_with_built_in_methods/

https://keras.io/guides/transfer_learning/

In [ ]:
#tf.config.run_functions_eagerly(True)
import tensorflow as tf
from tensorflow.python.ops.numpy_ops import np_config
np_config.enable_numpy_behavior()
from PIL import Image
import os
import pandas as pd
from typing import Union
import numpy as np
import time
import matplotlib.pyplot as plt
from keras import backend as K
from sklearn.metrics import confusion_matrix, f1_score
from keras.models import load_model
import matplotlib. image as image
from matplotlib import pyplot as plt
from random import random

seed_value = 42

tf.random.set_seed(seed_value)
np.random.seed(seed_value)


# Data loading

In [ ]:
! unzip /content/images.zip

In [ ]:
def create_directories(base_path, sub_folders):

    if not tf.io.gfile.exists(base_path):
        tf.io.gfile.mkdir(base_path)

    for folder in sub_folders:
        folder_path = f"{base_path}/{folder}"
        if not tf.io.gfile.exists(folder_path):
            tf.io.gfile.mkdir(folder_path)

base_paths = ['/content/train', '/content/test', '/content/augmentation']

sub_folders = ['HP', 'SSA']

for path in base_paths:
    create_directories(path, sub_folders)


In [ ]:
path = '/content/annotations.csv'
df = pd.read_csv(path)
csv = dict(df)
image_num = len(csv.get('Image Name'))


for i in range(image_num):
  if csv.get('Partition')[i] == "train":
    if csv.get('Majority Vote Label')[i] == "HP":
      src = '/content/images/' + csv.get('Image Name')[i]
      dst = '/content/train/HP/' + csv.get('Image Name')[i]
      tf.io.gfile.copy(src, dst, overwrite=True)
    else:
      src = '/content/images/' + csv.get('Image Name')[i]
      dst = '/content/train/SSA/' + csv.get('Image Name')[i]
      tf.io.gfile.copy(src, dst, overwrite=True)
  else:
    if csv.get('Majority Vote Label')[i] == "HP":
      src = '/content/images/' + csv.get('Image Name')[i]
      dst = '/content/test/HP/' + csv.get('Image Name')[i]
      tf.io.gfile.copy(src, dst, overwrite=True)
    else:
      src = '/content/images/' + csv.get('Image Name')[i]
      dst = '/content/test/SSA/' + csv.get('Image Name')[i]
      tf.io.gfile.copy(src, dst, overwrite=True)

# Data Augmentation

In [ ]:
path = '/content/annotations.csv'
df = pd.read_csv(path)
csv = dict(df)
image_num = len(csv.get('Image Name'))

for i in range(image_num):
    label = csv.get('Majority Vote Label')[i]
    src = '/content/images/' + csv.get('Image Name')[i]
    img = image.imread(src)
    img_aug = tf.image.rot90(tf.image.random_flip_up_down(img), k=round(random()*3))

    base_name = os.path.splitext(csv.get('Image Name')[i])[0]
    extension = os.path.splitext(csv.get('Image Name')[i])[1]
    augmented_name = base_name + '_aug' + extension

    if label == "HP":
        # Save augmented image
        tf.keras.utils.save_img('/content/augmentation/HP/' + augmented_name, img_aug, data_format=None, file_format=None, scale=True)
        # Save original image
        tf.keras.utils.save_img('/content/augmentation/HP/' + csv.get('Image Name')[i], img, data_format=None, file_format=None, scale=True)
    elif label == "SSA":
        # Save augmented image
        tf.keras.utils.save_img('/content/augmentation/SSA/' + augmented_name, img_aug, data_format=None, file_format=None, scale=True)
        # Save original image
        tf.keras.utils.save_img('/content/augmentation/SSA/' + csv.get('Image Name')[i], img, data_format=None, file_format=None, scale=True)


In [ ]:
train_dir = '/content/augmentation'

BATCH_SIZE=32
train_dataset = tf.keras.utils.image_dataset_from_directory(train_dir,shuffle=True,batch_size=BATCH_SIZE,image_size=(224,224))



test_dir = '/content/test'
test_dataset = tf.keras.utils.image_dataset_from_directory(test_dir,shuffle=True,batch_size=BATCH_SIZE,image_size=(224,224))




In [ ]:
train_dataset = train_dataset.map(lambda x, y: (tf.cast(x, tf.float32), tf.one_hot(y, 2)))
test_dataset = test_dataset.map(lambda x, y: (tf.cast(x, tf.float32), tf.one_hot(y, 2)))

# Loss unctions and evaluate function



In [ ]:
from sklearn.metrics import confusion_matrix, f1_score
import numpy as np
import tensorflow as tf

ALPHA = 0.5
TEMPERATURE = 4


def compute_student_loss_withoutKD(images, labels):
  subclass_logits = student_model(images, training=True)
  cross_entropy_loss_value = tf.nn.softmax_cross_entropy_with_logits(labels=labels, logits=subclass_logits)
  return cross_entropy_loss_value

def distillation_loss(teacher_logits: tf.Tensor, student_logits: tf.Tensor,
                      temperature: Union[float, tf.Tensor]):
  soft_targets = tf.nn.softmax(teacher_logits / temperature)
  return tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(
          soft_targets, student_logits / temperature)) * temperature ** 2


def compute_student_loss(images, labels):
  student_subclass_logits = student_model(images, training=True)
  teacher_subclass_logits = teacher_model(images, training=False)
  distillation_loss_value = distillation_loss(teacher_subclass_logits,student_subclass_logits,DISTILLATION_TEMPERATURE)
  cross_entropy_loss_value = tf.nn.softmax_cross_entropy_with_logits(labels, student_subclass_logits)
  loss= ALPHA*distillation_loss_value + (1-ALPHA)* cross_entropy_loss_value
  return loss

def compute_num_correct(model, images, labels):
  class_logits = model(images, training=False)
  value= tf.reduce_sum(
      tf.cast(tf.math.equal(tf.argmax(class_logits, -1), tf.argmax(labels, -1)),
              tf.float32)), tf.argmax(class_logits, -1), tf.argmax(labels, -1)
  return value

def compute_predictions(model, images):
    logits = model(images, training=False)
    return tf.argmax(logits, axis=1)

def compute_true_labels(labels):
    return tf.argmax(labels, axis=1)

def compute_teacher_loss(images, labels):
  subclass_logits = teacher_model(images, training=True)
  cross_entropy_loss_value = tf.nn.softmax_cross_entropy_with_logits(labels=labels, logits=subclass_logits)
  return cross_entropy_loss_value

def train_and_evaluate(model, compute_loss_fn):
    time_matrix = []
    f1_scores_label_0 = []  # Store F1 scores for label 0
    f1_scores_label_1 = []  # Store F1 scores for label 1
    macro_f1 = []

    for epoch in range(1, NUM_EPOCHS + 1):

        start_time = time.time()
        print('Epoch {}: '.format(epoch), end='')
        for images, labels in train_dataset:
            with tf.GradientTape() as tape:
                loss_value = compute_loss_fn(images, labels)
            grads = tape.gradient(loss_value, model.trainable_variables)
            optimizer.apply_gradients(zip(grads, model.trainable_variables))

        all_predictions = []
        all_labels = []
        for images, labels in test_dataset:
            predictions = compute_predictions(model, images)
            true_labels = compute_true_labels(labels)

            all_predictions.append(predictions)
            all_labels.append(true_labels)

        all_predictions = np.concatenate(all_predictions, axis=0)
        all_labels = np.concatenate(all_labels, axis=0)

        epoch_confusion_matrix = confusion_matrix(all_labels, all_predictions)

        epoch_f1_individual = f1_score(all_labels, all_predictions, average=None)
        f1_macro = f1_score(all_labels, all_predictions, average='macro')

        print("macro F1 Scores:", f1_macro)
        print(epoch_confusion_matrix)
        print("F1 Scores for each label:", epoch_f1_individual)

        epoch_time = time.time() - start_time
        time_matrix.append(epoch_time)

        f1_scores_label_0.append(epoch_f1_individual[0])
        f1_scores_label_1.append(epoch_f1_individual[1])
        macro_f1.append(f1_macro)

        print("Total time per epoch: {0:.2f} seconds".format(epoch_time))

    total_time = np.sum(time_matrix)
    print("Total time of training this model: {0:.2f} seconds".format(total_time))

    return f1_scores_label_0, f1_scores_label_1,macro_f1

# Train and Evalute

**Pre-trained ResNet50V2**

In [ ]:
from tensorflow.keras.layers import BatchNormalization

#Reference: https://keras.io/guides/transfer_learning/
input_size = (224,224,3)
teacher_resnet_50 = tf.keras.applications.resnet_v2.ResNet50V2(input_shape=input_size,
    include_top=False,
    weights='imagenet')

teacher_resnet_50.trainable=False

inputs = tf.keras.Input(shape=(224, 224, 3))
x = teacher_resnet_50(inputs, training=False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)

outputs = tf.keras.layers.Dense(2)(x)
teacher_model = tf.keras.Model(inputs, outputs)

optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)

teacher_model.compile(optimizer=optimizer)

teacher_model.summary()


In [ ]:
NUM_EPOCHS = 10
h1, h2,h3 = train_and_evaluate(teacher_model, compute_teacher_loss)

f1_1 = h1
f1_2 = h2
f1_3 = h3

plt.figure(figsize=(10, 10))
plt.plot(f1_1, label='F1: HP',linestyle='--', marker='o')
plt.plot(f1_2, label='F1: SSA',linestyle='--', marker='o')
plt.plot(f1_3, label='micro F1 ',linestyle='--', marker='o')

plt.ylabel('F1')
plt.ylim([0, 1])

plt.legend()
plt.title('F1 of Teacher Model')


plt.show()

**fine-tune teacher model**


In [ ]:
teacher_model.trainable = True

for layer in teacher_model.layers:
  layer.trainable = False
for layer in teacher_model.layers[-4:]:
  layer.trainable = True

optimizer=tf.keras.optimizers.Adam(1e-5)
teacher_model.compile(optimizer=optimizer)

teacher_model.summary()

NUM_EPOCHS = 25
h1_finetune, h2_finetune,h3_finetune = train_and_evaluate(teacher_model, compute_teacher_loss)

save the fine-tuned teacher model

In [ ]:
teacher_model.save('/content/teacher_model.h5')

Visualization for teacher model f1 score before and after fine-tune

In [ ]:
initial_epoch=10
f1_1 += h1_finetune
f1_2 += h2_finetune
f1_3 += h3_finetune

fig, ax = plt.subplots(figsize=(20, 10))

ax.plot(f1_1, linestyle='--', marker='o', label='F1 : HP')
ax.plot(f1_2, linestyle='--', marker='o', label='F1 : SSA')
ax.plot(f1_3, linestyle='--', marker='o', label='macro F1')

ax.axvline(x=initial_epoch, linestyle='--', label='Fine Tuning')

ax.set_title('Accuracy of Teacher Model')
ax.set_ylabel('F1')
ax.set_ylim([0, 1])
ax.legend()

plt.show()

Train and evaluate student model

In [ ]:
#Reference: https://keras.io/guides/transfer_learning/
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense

input_size = (224,224,3)
student_base_model= tf.keras.applications.MobileNetV2(input_shape=input_size,
                                              include_top=False,
                                              weights='imagenet')

student_base_model.trainable = False

inputs = tf.keras.Input(shape=(224,224,3))
x = student_base_model(inputs, training=False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = Dense(200, activation='relu')(x)
outputs = tf.keras.layers.Dense(2)(x)
student_model = tf.keras.Model(inputs, outputs)
student_model.summary()

student_model.compile(optimizer=tf.keras.optimizers.Adam(1e-3))

**Train the student model from scratch with 10 epochs and fine-tune with 25 epochs**

In [ ]:
NUM_EPOCHS= 10
optimizer = tf.optimizers.Adam(learning_rate=1e-3)

student_f1_1, student_f1_2,student_f1_3= train_and_evaluate(student_model,compute_student_loss_withoutKD)

In [ ]:
def fine_tune(model):
  model.trainable = True
  for layer in model.layers:
    layer.trainable = False
  for layer in model.layers[-2:]:
    layer.trainable = True

  optimizer = tf.optimizers.Adam(learning_rate=1e-4)
  model.compile(optimizer=optimizer)
  return model

In [ ]:
NUM_EPOCHS= 25
student_model = fine_tune(student_model)

student_f1_1_fine, student_f1_2_fine,student_f1_3_fine = train_and_evaluate(student_model,compute_student_loss_withoutKD)

**save the fine tuned student model**

In [ ]:
student_model.save('/content/student_model.h5')

**Train the student Model with knowledge Distillation**

In [ ]:
input_size = (224,224,3)
student_base_model= tf.keras.applications.MobileNetV2(input_shape=input_size,
                                              include_top=False,
                                              weights='imagenet')

student_base_model.trainable = False

inputs = tf.keras.Input(shape=(224,224,3))
x = student_base_model(inputs, training=False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
outputs = tf.keras.layers.Dense(2)(x)
student_model = tf.keras.Model(inputs, outputs)

optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
student_model.compile(optimizer=optimizer)


In [ ]:
student_model.summary()

In [ ]:
#Reference: https://keras.io/guides/transfer_learning/
input_size = (224,224,3)
teacher_resnet_50 = tf.keras.applications.resnet_v2.ResNet50V2(input_shape=input_size,
    include_top=False,
    weights='imagenet')

teacher_resnet_50.trainable=False

inputs = tf.keras.Input(shape=(224, 224, 3))
x = teacher_resnet_50(inputs, training=False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
outputs = tf.keras.layers.Dense(2)(x)
teacher_model = tf.keras.Model(inputs, outputs)

optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)

teacher_model.compile(optimizer=optimizer)

teacher_model.summary()

In [ ]:
teacher_model = tf.keras.models.load_model("teacher_model.h5")

In [ ]:
student_model = tf.keras.models.load_model("student_model.h5")
#student_model = fine_tune(student_model)
student_model.summary()

In [ ]:
ALPHA = 0.4
DISTILLATION_TEMPERATURE = 64
NUM_EPOCHS= 25
optimizer = tf.optimizers.Adam(learning_rate=1e-3)
student_model.summary()

In [ ]:
student_f1_1_fine_KD, student_f1_2_fine_KD,student_f1_3_fine_KD = train_and_evaluate(student_model,compute_student_loss)

In [ ]:
f1_1_student = student_f1_1 + student_f1_1_fine + student_f1_1_fine_KD
f1_2_student = student_f1_2 + student_f1_2_fine + student_f1_2_fine_KD
f1_3_student = student_f1_3 + student_f1_3_fine + student_f1_3_fine_KD
fig, ax = plt.subplots(figsize=(12, 8))

ax.plot(f1_1_student, linestyle='--', marker='o', label='F1 Score: HP')
ax.plot(f1_2_student, linestyle='--', marker='o', label='F1 Score: SSA')
ax.plot(f1_3_student, linestyle='--', marker='o', label='micro F1')

ax.axvline(x=initial_epoch, color='red', linestyle='--', label='Start Fine Tuning')
ax.axvline(x=35, color='gray', linestyle='--', label='Start KD')

ax.set_ylabel('Accuracy')
ax.set_ylim([0, 1])
ax.legend(loc='lower right')
ax.set_title('F1 score of Student Model')

plt.show()


# F1 Score with different Temperature

In [ ]:
Temperatures = [1,2,4,16,32,64]

ALPHA=0.5
student_f1_scores_1 = []
student_f1_scores_2 = []
student_f1_scores_3 = []

NUM_EPOCHS= 12


for t in Temperatures:
  DISTILLATION_TEMPERATURE = t
  tf.keras.backend.clear_session()
  input_size = (224,224,3)
  student_base_model= tf.keras.applications.MobileNetV2(input_shape=input_size,
                                                include_top=False,
                                                weights='imagenet')

  student_base_model.trainable = False

  inputs = tf.keras.Input(shape=(224,224,3))
  x = student_base_model(inputs, training=False)
  x = tf.keras.layers.GlobalAveragePooling2D()(x)
  x = tf.keras.layers.Dropout(0.2)(x)
  outputs = tf.keras.layers.Dense(2)(x)
  student_model = tf.keras.Model(inputs, outputs)
  student_model.compile(optimizer=tf.keras.optimizers.Adam(1e-3))
  optimizer = tf.optimizers.Adam(learning_rate=1e-3)
  student_model = fine_tune(student_model)
  student_model = load_model('/content/student_model.h5')
  f1_fine_1_KD,f1_fine_2_KD,f1_fine_3_KD = train_and_evaluate(student_model,compute_student_loss)
  student_f1_scores_1.append(f1_fine_1_KD)
  student_f1_scores_2.append(f1_fine_2_KD)
  student_f1_scores_3.append(f1_fine_3_KD)



In [ ]:
plt.figure(figsize=(15, 15))

plt.subplot(3, 1, 1)
for i in range(len(student_f1_scores_1)):
  plt.plot(student_f1_scores_1[i],linestyle='--', marker='o', label='Temperature={}'.format(Temperatures[i]))
  plt.ylabel('F1 Score')
  plt.legend(loc='lower left')
  plt.title('Student Model F1 Scores for class HP With Temperatures[1,2,4,16,32,64]')

plt.subplot(3, 1, 2)
for i in range(len(student_f1_scores_2)):
  plt.plot(student_f1_scores_2[i],linestyle='--', marker='o', label='Temperature={}'.format(Temperatures[i]))
  plt.legend(loc='lower right')
  plt.ylabel('F1 Score')
  plt.title('Student Model F1 Scores for class SSA With Temperatures[1,2,4,16,32,64]')

plt.subplot(3, 1, 3)
for i in range(len(student_f1_scores_3)):
  plt.plot(student_f1_scores_3[i],linestyle='--', marker='o', label='Temperature={}'.format(Temperatures[i]))
  plt.legend(loc='lower right')
  plt.ylabel('F1 Score')
  plt.title('Student Model F1 Scores With Temperatures[1,2,4,16,32,64]')


plt.show()

# implement with state of art model in paper 2 and 9

## paper 2 with early stopping knowledge distillation

In [ ]:
teacher_resnet_50 = tf.keras.applications.resnet_v2.ResNet50V2(
    include_top=False,
    weights='imagenet')

teacher_resnet_50.trainable=False

inputs = tf.keras.Input(shape=(224, 224, 3))
x = teacher_resnet_50(inputs, training=False)
x = tf.keras.layers.GlobalAveragePooling2D(dtype='float32')(x)
outputs = tf.keras.layers.Dense(2)(x)
teacher_model = tf.keras.Model(inputs, outputs)


optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)
teacher_model.compile(optimizer=optimizer
)


In [ ]:
input_size = (224,224,3)
student_base_model= tf.keras.applications.MobileNetV2(input_shape=input_size,
                                              include_top=False,
                                              weights='imagenet')

student_base_model.trainable = False

inputs = tf.keras.Input(shape=(224,224,3))
x = student_base_model(inputs, training=False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = Dense(200, activation='relu')(x)
outputs = tf.keras.layers.Dense(2)(x)
student_model = tf.keras.Model(inputs, outputs)

student_model.compile(optimizer=tf.keras.optimizers.Adam(1e-3))

In [ ]:
teacher_model = tf.keras.models.load_model("teacher_model.h5")
student_model = tf.keras.models.load_model("student_model.h5")

In [ ]:
from sklearn.metrics import confusion_matrix, f1_score
import numpy as np
import tensorflow as tf

ALPHA = 0.5
TEMPERATURE = 64


def distillation_loss(teacher_logits: tf.Tensor, student_logits: tf.Tensor,
                      temperature: Union[float, tf.Tensor]):
  soft_targets = tf.nn.softmax(teacher_logits / temperature)

  return tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(
          soft_targets, student_logits / temperature)) * temperature ** 2

def compute_student_loss_ES(images, labels,epochs):
  student_subclass_logits = student_model(images, training=True)
  teacher_subclass_logits = teacher_model(images, training=False)
  distillation_loss_value = distillation_loss(teacher_subclass_logits,student_subclass_logits,DISTILLATION_TEMPERATURE)
  cross_entropy_loss_value = tf.nn.softmax_cross_entropy_with_logits(labels, student_subclass_logits)
  if epochs <=10:
    loss = ALPHA*distillation_loss_value+(1-ALPHA)*cross_entropy_loss_value
  else:
    cross_entropy_loss_value = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels,student_subclass_logits))
    loss = cross_entropy_loss_value
  return loss


def compute_num_correct(model, images, labels):
  class_logits = model(images, training=False)
  value= tf.reduce_sum(
      tf.cast(tf.math.equal(tf.argmax(class_logits, -1), tf.argmax(labels, -1)),
              tf.float32)), tf.argmax(class_logits, -1), tf.argmax(labels, -1)
  return value

def compute_predictions(model, images):
    logits = model(images, training=False)
    return tf.argmax(logits, axis=1)

def compute_true_labels(labels):
    return tf.argmax(labels, axis=1)

def train_and_evaluate(model, compute_loss_fn):
    time_matrix = []
    f1_scores_label_0 = []  # Store F1 scores for label 0
    f1_scores_label_1 = []  # Store F1 scores for label 1
    macro_f1 = []

    for epoch in range(1, NUM_EPOCHS + 1):

        start_time = time.time()
        print('Epoch {}: '.format(epoch), end='')
        for images, labels in train_dataset:
            with tf.GradientTape() as tape:
                  loss_value = compute_loss_fn(images,labels,epoch)
            grads = tape.gradient(loss_value, model.trainable_variables)
            optimizer.apply_gradients(zip(grads, model.trainable_variables))

        all_predictions = []
        all_labels = []
        for images, labels in test_dataset:
            predictions = compute_predictions(model, images)
            true_labels = compute_true_labels(labels)

            all_predictions.append(predictions)
            all_labels.append(true_labels)

        all_predictions = np.concatenate(all_predictions, axis=0)
        all_labels = np.concatenate(all_labels, axis=0)

        epoch_confusion_matrix = confusion_matrix(all_labels, all_predictions)

        epoch_f1_individual = f1_score(all_labels, all_predictions, average=None)
        f1_macro = f1_score(all_labels, all_predictions, average='macro')

        print("macro F1 Scores:", f1_macro)
        print(epoch_confusion_matrix)
        print("F1 Scores for each label:", epoch_f1_individual)

        epoch_time = time.time() - start_time
        time_matrix.append(epoch_time)

        f1_scores_label_0.append(epoch_f1_individual[0])
        f1_scores_label_1.append(epoch_f1_individual[1])
        macro_f1.append(f1_macro)

        print("Total time per epoch: {0:.2f} seconds".format(epoch_time))

    total_time = np.sum(time_matrix)
    print("Total time of training this model: {0:.2f} seconds".format(total_time))

    return f1_scores_label_0, f1_scores_label_1,macro_f1

In [ ]:
NUM_EPOCHS= 25
train_and_evaluate(student_model, compute_student_loss_ES)

## paper 9 with teacher assistant model

The TA model is choosen as EffiecientNetB0, which is a model designed to be both smaller in size and more efficient in terms of performance and has a model size between ResNetV2 and MobileNetV2.

In [ ]:
# Load the EfficientNetB0 model as the TA model
teacher_efficientnet = tf.keras.applications.EfficientNetB0(
    include_top=False,
    weights='imagenet'
)

teacher_efficientnet.trainable = False

inputs = tf.keras.Input(shape=(224, 224, 3))
x = teacher_efficientnet(inputs, training=False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
outputs = tf.keras.layers.Dense(2)(x)
ta_model = tf.keras.Model(inputs, outputs)

# Compile the TA model
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)
ta_model.compile(optimizer=optimizer)


In [ ]:
input_size = (224,224,3)
student_base_model= tf.keras.applications.MobileNetV2(input_shape=input_size,
                                              include_top=False,
                                              weights='imagenet')

student_base_model.trainable = False

inputs = tf.keras.Input(shape=(224,224,3))
x = student_base_model(inputs, training=False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = Dense(200, activation='relu')(x)
outputs = tf.keras.layers.Dense(2)(x)
student_model = tf.keras.Model(inputs, outputs)
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)

student_model.compile(optimizer=tf.keras.optimizers.Adam(1e-3))

In [ ]:
from sklearn.metrics import confusion_matrix, f1_score
import numpy as np
import tensorflow as tf

ALPHA = 0.5
TEMPERATURE = 4

def compute_ta_loss_finetune(images, labels):
  subclass_logits = ta_model(images, training=True)
  cross_entropy_loss_value = tf.nn.softmax_cross_entropy_with_logits(labels=labels, logits=subclass_logits)
  return cross_entropy_loss_value

def distillation_loss(teacher_logits: tf.Tensor, student_logits: tf.Tensor,
                      temperature: Union[float, tf.Tensor]):
  soft_targets = tf.nn.softmax(teacher_logits / temperature)

  return tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(
          soft_targets, student_logits / temperature)) * temperature ** 2

def compute_ta_loss(images, labels):
  ta_subclass_logits = ta_model(images, training=True)
  teacher_subclass_logits = teacher_model(images, training=False)
  distillation_loss_value = distillation_loss(teacher_subclass_logits,ta_subclass_logits,DISTILLATION_TEMPERATURE)
  cross_entropy_loss_value = tf.nn.softmax_cross_entropy_with_logits(labels, ta_subclass_logits)
  loss= ALPHA*distillation_loss_value + (1-ALPHA)* cross_entropy_loss_value
  return loss


def compute_student_loss(images, labels):
  student_subclass_logits = student_model(images, training=True)
  teacher_subclass_logits = ta_model(images, training=False)
  distillation_loss_value = distillation_loss(teacher_subclass_logits,student_subclass_logits,DISTILLATION_TEMPERATURE)
  cross_entropy_loss_value = tf.nn.softmax_cross_entropy_with_logits(labels, student_subclass_logits)
  loss= ALPHA*distillation_loss_value + (1-ALPHA)* cross_entropy_loss_value
  return loss


def compute_predictions(model, images):
    logits = model(images, training=False)
    return tf.argmax(logits, axis=1)

def compute_true_labels(labels):
    return tf.argmax(labels, axis=1)

def train_and_evaluate(model, compute_loss_fn):
    time_matrix = []
    f1_scores_label_0 = []  # Store F1 scores for label 0
    f1_scores_label_1 = []  # Store F1 scores for label 1
    macro_f1 = []

    for epoch in range(1, NUM_EPOCHS + 1):

        start_time = time.time()
        print('Epoch {}: '.format(epoch), end='')
        for images, labels in train_dataset:
            with tf.GradientTape() as tape:
                loss_value = compute_loss_fn(images, labels)
            #print(loss_value)
            grads = tape.gradient(loss_value, model.trainable_variables)
            optimizer.apply_gradients(zip(grads, model.trainable_variables))

        all_predictions = []
        all_labels = []
        for images, labels in test_dataset:
            predictions = compute_predictions(model, images)
            true_labels = compute_true_labels(labels)

            all_predictions.append(predictions)
            all_labels.append(true_labels)

        all_predictions = np.concatenate(all_predictions, axis=0)
        all_labels = np.concatenate(all_labels, axis=0)

        epoch_confusion_matrix = confusion_matrix(all_labels, all_predictions)

        epoch_f1_individual = f1_score(all_labels, all_predictions, average=None)
        f1_macro = f1_score(all_labels, all_predictions, average='macro')

        print("macro F1 Scores:", f1_macro)
        print(epoch_confusion_matrix)
        print("F1 Scores for each label:", epoch_f1_individual)

        epoch_time = time.time() - start_time
        time_matrix.append(epoch_time)

        f1_scores_label_0.append(epoch_f1_individual[0])
        f1_scores_label_1.append(epoch_f1_individual[1])
        macro_f1.append(f1_macro)

        print("Total time per epoch: {0:.2f} seconds".format(epoch_time))

    total_time = np.sum(time_matrix)
    print("Total time of training this model: {0:.2f} seconds".format(total_time))

    return f1_scores_label_0, f1_scores_label_1,macro_f1

In [ ]:
teacher_model = tf.keras.models.load_model("teacher_model.h5")
student_model = tf.keras.models.load_model("student_model.h5")

**fine tune TA**

In [ ]:
ta_model.trainable = True

for layer in ta_model.layers:
  layer.trainable = False
for layer in ta_model.layers[-3:]:
  layer.trainable = True

optimizer=tf.keras.optimizers.Adam(1e-3)
ta_model.compile(optimizer=optimizer)


NUM_EPOCHS = 25
train_and_evaluate(ta_model, compute_ta_loss_finetune)

**KD ta model**

In [ ]:
train_and_evaluate(ta_model, compute_ta_loss)

**KD student model**

In [ ]:
train_and_evaluate(student_model, compute_student_loss)

# flops

In [ ]:
def get_flops(teacher_model):
    session = tf.compat.v1.Session()
    graph = tf.compat.v1.get_default_graph()

    with graph.as_default():
        with session.as_default():
            model = tf.keras.models.clone_model(teacher_model)
            run_meta = tf.compat.v1.RunMetadata()
            opts = tf.compat.v1.profiler.ProfileOptionBuilder.float_operation()

            flops = tf.compat.v1.profiler.profile(graph=graph,
                                                  run_meta=run_meta, cmd='op', options=opts)
            return flops.total_float_ops

print(f"The number of FLOPs of the teacher model: {get_flops(teacher_model)}")
print(f"The number of FLOPs of the student model: {get_flops(student_model)}")